In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_log_error

import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('D:/IMS Proschool/Business Analytics/Practice/Self/E-Commerce_Participants_Data/Train.csv')
test = pd.read_csv('D:/IMS Proschool/Business Analytics/Practice/Self/E-Commerce_Participants_Data/Test.csv')

In [4]:
train.shape

(2452, 8)

In [5]:
test.shape

(1051, 7)

In [12]:
data = pd.concat([train,test], axis=0, ignore_index=True)
data.shape

(3503, 8)

In [13]:
data.head()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
0,P-2610,B-659,bags wallets belts,bags,hand bags,4.3,2/3/2017,291.0
1,P-2453,B-3078,clothing,women s clothing,western wear,3.1,7/1/2015,897.0
2,P-6802,B-1810,home decor festive needs,showpieces,ethnic,3.5,1/12/2019,792.0
3,P-4452,B-3078,beauty and personal care,eye care,h2o plus eye care,4.0,12/12/2014,837.0
4,P-8454,B-3078,clothing,men s clothing,t shirts,4.3,12/12/2013,470.0


In [14]:
data.tail()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
3498,P-9758,B-2543,sports fitness,outdoor adventure,camping hiking,2.0,28/2/2014,NaN
3499,P-11898,B-3197,jewellery,necklaces chains,necklaces,4.7,1/12/2019,NaN
3500,P-904,B-133,automotive,accessories spare parts,car interior exterior,3.5,1/12/2014,NaN
3501,P-1714,B-358,kitchen dining,lighting,bulbs,2.4,23/6/2013,NaN
3502,P-620,B-133,automotive,accessories spare parts,car interior exterior,3.1,1/12/2012,NaN


In [15]:
data_features = data.drop(['Product','Date'], axis=1)

In [18]:
ct = ColumnTransformer([('encoder', OneHotEncoder(), [0,1,2,3,4])], remainder='passthrough') 
data_features = ct.fit_transform(data_features)
data_features

array([[  1.,   0.,   1., ...,   0.,   0., 291.],
       [  1.,   0.,   1., ...,   0.,   0., 897.],
       [  1.,   0.,   1., ...,   0.,   0., 792.],
       ...,
       [  1.,   0.,   1., ...,   0.,   0.,  nan],
       [  1.,   0.,   1., ...,   0.,   0.,  nan],
       [  1.,   0.,   1., ...,   0.,   0.,  nan]])

In [22]:
data_encoded = pd.DataFrame(data_features)
data_encoded = data_encoded.rename(columns={1991:"Selling_Price"})
data_encoded.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,...,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,Selling_Price
0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,291.0
1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,897.0
2,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,792.0
3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,837.0
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,470.0


In [23]:
data_encoded.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,...,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,Selling_Price
3498,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3499,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
3500,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3501,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3502,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [25]:
train_features = data_encoded[data_encoded['Selling_Price'].isna() == False]
test_features = data_encoded[data_encoded['Selling_Price'].isna() == True]
train_label = train_features['Selling_Price']

In [26]:
train_features = train_features.drop(['Selling_Price'], axis=1)
test_features = test_features.drop(['Selling_Price'], axis=1)

In [67]:
rf = RandomForestRegressor()
etr = ExtraTreesRegressor()
abr = AdaBoostRegressor()
xgb = XGBRegressor()
knr = KNeighborsRegressor()
cbr = CatBoostRegressor()

In [29]:
rf_score = cross_val_score(rf, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

In [30]:
rf_score.mean()

-0.49492311619165774

In [31]:
etr_score = cross_val_score(etr, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

In [32]:
etr_score.mean()

-0.5746822439682844

In [33]:
abr_score = cross_val_score(abr, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

In [34]:
abr_score.mean()

-1.820335513542662

In [35]:
xgb_score = cross_val_score(xgb, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

[14:00:43] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:01:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:01:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:01:37] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:01:55] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [36]:
xgb_score.mean()

-0.8424749133157686

In [37]:
knr_score = cross_val_score(knr, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

In [38]:
knr_score.mean()

-0.7836167735092796

In [39]:
cbr_score = cross_val_score(cbr, train_features, train_label, cv=5, scoring='neg_mean_squared_log_error')

Learning rate set to 0.044001
0:	learn: 6556.7182663	total: 260ms	remaining: 4m 20s
1:	learn: 6353.0139037	total: 287ms	remaining: 2m 22s
2:	learn: 6158.8699813	total: 298ms	remaining: 1m 39s
3:	learn: 5977.4095224	total: 310ms	remaining: 1m 17s
4:	learn: 5802.3396120	total: 318ms	remaining: 1m 3s
5:	learn: 5633.8869684	total: 326ms	remaining: 54s
6:	learn: 5475.5912696	total: 334ms	remaining: 47.4s
7:	learn: 5328.8396980	total: 342ms	remaining: 42.4s
8:	learn: 5186.2939003	total: 350ms	remaining: 38.5s
9:	learn: 5051.1934541	total: 358ms	remaining: 35.4s
10:	learn: 4923.9406951	total: 366ms	remaining: 32.9s
11:	learn: 4803.2540462	total: 374ms	remaining: 30.8s
12:	learn: 4687.9346893	total: 382ms	remaining: 29s
13:	learn: 4582.7766733	total: 390ms	remaining: 27.4s
14:	learn: 4480.4895574	total: 397ms	remaining: 26.1s
15:	learn: 4383.9766222	total: 408ms	remaining: 25.1s
16:	learn: 4285.4530998	total: 420ms	remaining: 24.3s
17:	learn: 4201.4023155	total: 429ms	remaining: 23.4s
18:	lear

152:	learn: 2297.5949513	total: 1.63s	remaining: 9.03s
153:	learn: 2294.9623513	total: 1.64s	remaining: 9.01s
154:	learn: 2290.1445180	total: 1.65s	remaining: 8.99s
155:	learn: 2286.9322265	total: 1.66s	remaining: 8.99s
156:	learn: 2284.7025819	total: 1.67s	remaining: 8.96s
157:	learn: 2282.1427061	total: 1.68s	remaining: 8.94s
158:	learn: 2278.0013479	total: 1.69s	remaining: 8.92s
159:	learn: 2270.8660642	total: 1.69s	remaining: 8.9s
160:	learn: 2268.7635732	total: 1.7s	remaining: 8.87s
161:	learn: 2264.8053129	total: 1.71s	remaining: 8.85s
162:	learn: 2262.4069178	total: 1.72s	remaining: 8.83s
163:	learn: 2259.4637812	total: 1.73s	remaining: 8.81s
164:	learn: 2256.6750372	total: 1.74s	remaining: 8.79s
165:	learn: 2248.1789415	total: 1.74s	remaining: 8.76s
166:	learn: 2245.2365963	total: 1.75s	remaining: 8.74s
167:	learn: 2242.3149089	total: 1.76s	remaining: 8.72s
168:	learn: 2239.4287621	total: 1.77s	remaining: 8.69s
169:	learn: 2237.0852868	total: 1.78s	remaining: 8.67s
170:	learn: 

322:	learn: 1845.8557995	total: 3.05s	remaining: 6.39s
323:	learn: 1844.7094511	total: 3.06s	remaining: 6.38s
324:	learn: 1842.4093136	total: 3.06s	remaining: 6.37s
325:	learn: 1841.2764021	total: 3.08s	remaining: 6.36s
326:	learn: 1840.1458125	total: 3.08s	remaining: 6.35s
327:	learn: 1839.0130179	total: 3.09s	remaining: 6.34s
328:	learn: 1837.9055760	total: 3.1s	remaining: 6.33s
329:	learn: 1836.8009514	total: 3.11s	remaining: 6.31s
330:	learn: 1833.7833692	total: 3.12s	remaining: 6.3s
331:	learn: 1832.6891726	total: 3.13s	remaining: 6.29s
332:	learn: 1831.6022606	total: 3.13s	remaining: 6.28s
333:	learn: 1829.1296856	total: 3.14s	remaining: 6.26s
334:	learn: 1825.6517733	total: 3.15s	remaining: 6.25s
335:	learn: 1824.5689261	total: 3.16s	remaining: 6.24s
336:	learn: 1823.4885650	total: 3.17s	remaining: 6.23s
337:	learn: 1822.4228091	total: 3.17s	remaining: 6.21s
338:	learn: 1820.6314009	total: 3.18s	remaining: 6.21s
339:	learn: 1819.5643593	total: 3.19s	remaining: 6.19s
340:	learn: 

474:	learn: 1654.4513967	total: 4.3s	remaining: 4.76s
475:	learn: 1652.4895585	total: 4.32s	remaining: 4.75s
476:	learn: 1651.4637280	total: 4.33s	remaining: 4.74s
477:	learn: 1650.8158905	total: 4.33s	remaining: 4.73s
478:	learn: 1650.1729562	total: 4.34s	remaining: 4.72s
479:	learn: 1648.0211903	total: 4.35s	remaining: 4.71s
480:	learn: 1647.3803917	total: 4.36s	remaining: 4.7s
481:	learn: 1646.7420918	total: 4.37s	remaining: 4.69s
482:	learn: 1646.1045000	total: 4.38s	remaining: 4.68s
483:	learn: 1645.0908037	total: 4.38s	remaining: 4.67s
484:	learn: 1643.3917257	total: 4.39s	remaining: 4.66s
485:	learn: 1642.7602326	total: 4.4s	remaining: 4.65s
486:	learn: 1642.1314621	total: 4.41s	remaining: 4.64s
487:	learn: 1640.4110829	total: 4.42s	remaining: 4.63s
488:	learn: 1639.7836647	total: 4.42s	remaining: 4.62s
489:	learn: 1639.1600984	total: 4.43s	remaining: 4.61s
490:	learn: 1638.5399179	total: 4.44s	remaining: 4.6s
491:	learn: 1637.9188354	total: 4.45s	remaining: 4.59s
492:	learn: 16

636:	learn: 1515.0612881	total: 5.72s	remaining: 3.26s
637:	learn: 1514.6461316	total: 5.72s	remaining: 3.25s
638:	learn: 1514.1977085	total: 5.73s	remaining: 3.24s
639:	learn: 1513.7542568	total: 5.74s	remaining: 3.23s
640:	learn: 1513.3432107	total: 5.75s	remaining: 3.22s
641:	learn: 1512.9332649	total: 5.76s	remaining: 3.21s
642:	learn: 1512.5114483	total: 5.77s	remaining: 3.2s
643:	learn: 1510.6444270	total: 5.77s	remaining: 3.19s
644:	learn: 1510.2118575	total: 5.78s	remaining: 3.18s
645:	learn: 1509.7807128	total: 5.79s	remaining: 3.17s
646:	learn: 1509.1231834	total: 5.8s	remaining: 3.16s
647:	learn: 1508.7012490	total: 5.8s	remaining: 3.15s
648:	learn: 1508.2811506	total: 5.81s	remaining: 3.14s
649:	learn: 1507.8651523	total: 5.82s	remaining: 3.13s
650:	learn: 1507.4553880	total: 5.83s	remaining: 3.13s
651:	learn: 1507.0503141	total: 5.84s	remaining: 3.12s
652:	learn: 1506.6464064	total: 5.85s	remaining: 3.11s
653:	learn: 1506.0067801	total: 5.85s	remaining: 3.1s
654:	learn: 15

...

438:	learn: 1798.2574861	total: 3.75s	remaining: 4.8s
439:	learn: 1797.3169579	total: 3.76s	remaining: 4.79s
440:	learn: 1795.0807181	total: 3.77s	remaining: 4.78s
441:	learn: 1794.1454789	total: 3.78s	remaining: 4.78s
442:	learn: 1793.2138600	total: 3.79s	remaining: 4.77s
443:	learn: 1792.3290317	total: 3.8s	remaining: 4.76s
444:	learn: 1790.1166612	total: 3.81s	remaining: 4.75s
445:	learn: 1789.1986804	total: 3.82s	remaining: 4.74s
446:	learn: 1788.2843377	total: 3.82s	remaining: 4.73s
447:	learn: 1787.4193630	total: 3.83s	remaining: 4.72s
448:	learn: 1786.5155834	total: 3.84s	remaining: 4.71s
449:	learn: 1784.1611385	total: 3.85s	remaining: 4.7s
450:	learn: 1783.2633014	total: 3.86s	remaining: 4.69s
451:	learn: 1782.3689534	total: 3.86s	remaining: 4.68s
452:	learn: 1781.4839959	total: 3.87s	remaining: 4.68s
453:	learn: 1780.6003365	total: 3.88s	remaining: 4.67s
454:	learn: 1779.7222541	total: 3.89s	remaining: 4.66s
455:	learn: 1778.3247703	total: 3.9s	remaining: 4.65s
456:	learn: 17

594:	learn: 1618.7395597	total: 5.06s	remaining: 3.45s
595:	learn: 1618.1549597	total: 5.07s	remaining: 3.44s
596:	learn: 1617.5717204	total: 5.08s	remaining: 3.43s
597:	learn: 1615.9084644	total: 5.09s	remaining: 3.42s
598:	learn: 1615.3299584	total: 5.1s	remaining: 3.41s
599:	learn: 1613.7479201	total: 5.11s	remaining: 3.4s
600:	learn: 1612.8350512	total: 5.12s	remaining: 3.4s
601:	learn: 1611.9396928	total: 5.12s	remaining: 3.39s
602:	learn: 1611.3618997	total: 5.13s	remaining: 3.38s
603:	learn: 1609.4773284	total: 5.14s	remaining: 3.37s
604:	learn: 1608.8082600	total: 5.15s	remaining: 3.36s
605:	learn: 1607.9402000	total: 5.16s	remaining: 3.35s
606:	learn: 1607.3662523	total: 5.16s	remaining: 3.34s
607:	learn: 1606.7953066	total: 5.17s	remaining: 3.33s
608:	learn: 1606.2273568	total: 5.18s	remaining: 3.33s
609:	learn: 1605.2905383	total: 5.19s	remaining: 3.32s
610:	learn: 1604.7244615	total: 5.2s	remaining: 3.31s
611:	learn: 1604.1624324	total: 5.2s	remaining: 3.3s
612:	learn: 1603

765:	learn: 1470.9067647	total: 6.51s	remaining: 1.99s
766:	learn: 1470.4828269	total: 6.52s	remaining: 1.98s
767:	learn: 1470.0678640	total: 6.53s	remaining: 1.97s
768:	learn: 1469.6483993	total: 6.54s	remaining: 1.96s
769:	learn: 1469.2342073	total: 6.55s	remaining: 1.96s
770:	learn: 1468.7399333	total: 6.56s	remaining: 1.95s
771:	learn: 1467.4686994	total: 6.57s	remaining: 1.94s
772:	learn: 1467.0558630	total: 6.57s	remaining: 1.93s
773:	learn: 1465.9891402	total: 6.58s	remaining: 1.92s
774:	learn: 1465.5799323	total: 6.59s	remaining: 1.91s
775:	learn: 1464.0908250	total: 6.6s	remaining: 1.9s
776:	learn: 1463.4473131	total: 6.61s	remaining: 1.9s
777:	learn: 1462.8696356	total: 6.61s	remaining: 1.89s
778:	learn: 1462.4629616	total: 6.62s	remaining: 1.88s
779:	learn: 1462.0581386	total: 6.63s	remaining: 1.87s
780:	learn: 1461.6538879	total: 6.64s	remaining: 1.86s
781:	learn: 1461.2517157	total: 6.65s	remaining: 1.85s
782:	learn: 1460.5393300	total: 6.66s	remaining: 1.84s
783:	learn: 1

936:	learn: 1368.1048387	total: 7.99s	remaining: 537ms
937:	learn: 1367.3209676	total: 8s	remaining: 529ms
938:	learn: 1366.3971621	total: 8.01s	remaining: 520ms
939:	learn: 1366.0730759	total: 8.02s	remaining: 512ms
940:	learn: 1365.5612096	total: 8.02s	remaining: 503ms
941:	learn: 1365.2582986	total: 8.03s	remaining: 495ms
942:	learn: 1364.9381588	total: 8.04s	remaining: 486ms
943:	learn: 1364.6223333	total: 8.05s	remaining: 477ms
944:	learn: 1364.3056199	total: 8.06s	remaining: 469ms
945:	learn: 1363.9938055	total: 8.06s	remaining: 460ms
946:	learn: 1363.6830105	total: 8.07s	remaining: 452ms
947:	learn: 1363.3774416	total: 8.08s	remaining: 443ms
948:	learn: 1363.0723871	total: 8.09s	remaining: 435ms
949:	learn: 1362.1238403	total: 8.1s	remaining: 426ms
950:	learn: 1361.4299426	total: 8.1s	remaining: 418ms
951:	learn: 1361.1298470	total: 8.11s	remaining: 409ms
952:	learn: 1359.8922221	total: 8.12s	remaining: 400ms
953:	learn: 1359.0025953	total: 8.13s	remaining: 392ms
954:	learn: 135

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [41]:
X_train, X_val, y_train, y_val = train_test_split(train_features, train_label, train_size=0.8, random_state=1)

In [42]:
rf

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [47]:
params = {'bootstrap': [True,False],
          'criterion': ['mse','mae'],
          'max_depth': [None,2,4,6,8],
          'max_features': [4,6,8,10],
          'min_samples_split': [2,4,5,6],
          'min_samples_leaf': [1,2,4,6],
          'max_leaf_nodes': [None,2,4,6],
          'n_estimators': [10,30,100]}

In [69]:
params2 = {'bootstrap': [False],
          'criterion': ['mae'],
          #'max_depth': [None,2,4,6,8],
          'max_features': [9,10,11,12],
          'min_samples_split': [4,5,6,7,8],
          #'min_samples_leaf': [1,2,4,6],
          #'max_leaf_nodes': [None,2,4,6],
          'n_estimators': [90,100,150,200]}

In [70]:
grid_search2 = GridSearchCV(rf, param_grid=params2, cv=3, scoring='neg_mean_squared_log_error', verbose=1, n_jobs=-1)

In [49]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 15360 candidates, totalling 46080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 11242 tasks      |

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, rand...
             iid='deprecated', n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         '

In [71]:
grid_search2.fit(X_train, y_train)

Fitting 3 folds for each of 80 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 21.3min


KeyboardInterrupt: 

In [52]:
y_pred = grid_search.predict(X_val)

In [52]:
y_pred2 = grid_search2.predict(X_val)

In [53]:
mean_squared_log_error(y_pred,y_val)

0.6551927895185529

In [53]:
mean_squared_log_error(y_pred2,y_val)

0.6551927895185529

In [54]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features=10, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=6, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [54]:
grid_search2.best_estimator_

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features=10, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=6, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [56]:
pars = { "bootstrap":False, "criterion":"mae", "max_features":10, "min_samples_split":6}
rf = RandomForestRegressor(**pars)

In [57]:
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features=10, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=6, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [59]:
y_pred2 = rf.predict(X_val)
mean_squared_log_error(y_pred2,y_val)

0.522483192042556

In [63]:
pars2 = { "bootstrap":False, "criterion":"mae", "max_features":6, "min_samples_split":2}
rf = RandomForestRegressor(**pars2)

In [64]:
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mae',
                      max_depth=None, max_features=6, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [65]:
y_pred3 = rf.predict(X_val)
mean_squared_log_error(y_pred3,y_val)

0.5939856430073344